# Zero Shot Classification

In [14]:
%pip install -q transformers torch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load Dataset

In [17]:
from datasets import load_dataset

# load and download the dataset from huggingface
dataset = load_dataset("LittleFish-Coder/Fake-News-Detection-Challenge-KDD-2020", download_mode="reuse_cache_if_exists", cache_dir="dataset")

Generating test split: 100%|██████████| 499/499 [00:00<00:00, 16855.04 examples/s]


In [18]:
# load the test dataset
test_dataset = dataset["test"]

In [19]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 499
})

# Load Model

In [20]:
# Load Model
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

/home/littlefish/miniconda3/envs/fakenews/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Test Model

In [21]:
id2label = {1: 'fake', 0: 'real'}
label2id = {'fake': 1, 'real': 0}

In [22]:
candidate_labels = ["fake", "real"]

In [23]:
results = classifier(test_dataset['text'], candidate_labels)

In [24]:
pred_labels = []

for result in results:
    pred_labels.append(label2id[result['labels'][0]])

# Evaluate Model

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Ground Truth Labels
real_labels = test_dataset['label']

pred_labels

accuracy = accuracy_score(real_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(real_labels, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy: 0.62
Precision: 0.67
Recall: 0.62
F1-score: 0.53
